In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
from scipy.special import sph_harm 
from scipy.special import legendre
from scipy import integrate
from scipy.integrate import dblquad
from scipy.integrate import quad
from math import factorial
%run -i func_sph_harm.ipynb

The equation used for projection of the spherical harmonics: 
$$
\begin{equation}
    \psi_{nm}=\int ^{2\pi}_0 \int^\pi_0 \psi(\theta, \phi)(Y^m_n(\theta,\phi))^\ast \sin \theta d\theta d\phi
\end{equation}
$$

In [2]:
def projection(f_tp, theta, phi, lMax): # Projection onto sph_harm: takes f(theta,phi) as input and returns f_{nm}
    t, p = np.meshgrid (theta, phi)
    f_nm = np.zeros((lMax, lMax), dtype=complex)

    for m in range (lMax): 
        #Y_nm = sphericalHarmonic(np.array([m, lMax]), np.array([0, lMax]), theta, phi).reshape(N_t, N_p).T
        for n in range (m, lMax):
            Y_nm = sphericalHarmonic(n, m, theta, phi).reshape(N_t, N_p).T
            Y_ast = np.conj(Y_nm)
            f_nm[n,m] = integrate.simpson(integrate.simpson(f_tp*Y_ast*np.sin(theta), theta), phi)
    '''f_nm shape is (lMax, lMax)'''
    return f_nm

The equation used for reconstruction:
$$
\begin{equation}
    \psi (\theta, \phi) = \sum ^{N}_{\lvert m \rvert = 0} \sum ^{N}_{n=\lvert m \rvert} \psi_{nm} Y^m_n(\theta, \phi)
\end{equation}
$$

In [3]:
def reconstruction(f_nm, theta, phi, lMax):  #Reconstruction from sph_harm, takes f_{nm} and returns f(theta,phi)
    f_TP = np.zeros((N_p, N_t), dtype=complex)
    #double sum
    for m in range(lMax):
        #Y_nm = sphericalHarmonic(np.array([m, lMax]), np.array([0, lMax]), theta, phi)
        for n in range(m, lMax):
            #Y_nm = sph_harm (m, n, p.reshape(-1), t.reshape(-1)).reshape(N_p, N_t)
            Y_nm = sphericalHarmonic(n, m, theta, phi).reshape(N_t, N_p).T
            f_TP += np.real(f_nm[n,m]*Y_nm)
        '''f_TP shape (N_p, N_t)'''
    return f_TP

In [5]:
def Ysin(m,n,theta,phi):
    mp1 = np.sqrt(((2.*n+1.)*(n-m)*(n-m-1.))/(2.*n-1.))
    mm1 = np.sqrt((2.*n+1.)*(n+m-1)*(n+m)/(2.*n-1.))
    t, p = np.meshgrid(theta, phi)
    #A = np.exp(-1j*p).reshape(-1)*sph_harm(m+1, n-1, p.reshape(-1), t.reshape(-1))*mp1
    A = np.exp(-1j*(p.T))*(sphericalHarmonic(n-1, m+1, theta, phi).reshape(N_t, N_p))*mp1
    #B = np.exp(1j*p).reshape(-1)*sph_harm(m-1, n-1, p.reshape(-1), t.reshape(-1))*mm1
    B = np.exp(1j*(p.T))*(sphericalHarmonic(n-1, m-1, theta, phi).reshape(N_t, N_p))*mm1
    #sp = sphericalHarmonic(2, 0, theta, phi, normalized=True).reshape(N_t,N_p) #sphericalHarmonic(n,m,theta,phi)
    #sp_a = (1/4)*np.sqrt(5/np.pi)*(3*np.cos(t)**2-1).reshape(N_p, N_t).T
    '''sp and sp_a both have shape (N_t, N_p)'''
    
    '''A, B has the shape (N_t,N_p)'''
    if (m+2)<=n:
        Ysin = -1*(A+B)/(2*m)
    else:
        Ysin = -1*B/(2*m) 
        '''Ysin shape(N_t,N_p)'''
    return Ysin.reshape(N_t,N_p).T

In [6]:
#f(theta,phi)/(sin\theta)
def fsin(f, theta, phi, lMax):
    output = np.zeros((N_p, N_t))
    t, p = np.meshgrid(theta, phi)
    f_nm = projection(f, theta, phi, lMax)
    #f = np.zeros((N_p, N_t))
    #Ysin = np.zeros((N_t, N_p), dtype=complex)
    for m in range(1, lMax):
        for n in range(m, lMax):
            Y = Ysin(m,n,theta,phi)
            output += 2.*np.real(f_nm[n,m]*Y)
    return output

$$
\zeta=-\frac{\partial V}{\partial \cos\theta} - \frac{1}{\sin^2\theta}\frac{\partial U}{\partial \phi} = \frac{1}{\sin\theta}\frac{\partial V}{\partial \theta} - \frac{1}{\sin^2\theta}\frac{\partial U}{\partial \phi}
$$

In [7]:
# to calculate zeta_tp
def zeta(U_tp, V_tp, theta, phi, lMax): 
    t, p = np.meshgrid(theta, phi)
    dvdt = np.gradient(V_tp, theta, axis=1)
    usin = fsin(U_tp, theta, phi, lMax)
    dusindp = np.gradient(usin, phi, axis=0)
    zeta_tp = (1/r)*fsin(dvdt-dusindp, theta, phi, lMax)
    #zeta_nm = projection(zeta_tp, theta, phi, lMax)
    return zeta_tp

$$
p_{nm}=\frac{-1}{n(n+1)}(\nabla^2P)_{nm}
$$

In [8]:
# to calculate p_nm
def pressure(U_TP, V_TP, zeta_tp, omega, theta, phi, lMax): 
    t, p = np.meshgrid(theta, phi)
    U1_tp = V_TP*zeta_tp+2*omega*np.cos(t)*V_TP #U'(theta, phi)
    V1_tp = -1*U_TP*zeta_tp-2*omega*np.cos(t)*U_TP #V'(theta, phi)
    du1dt = np.gradient(U1_tp, theta, axis=1)
    divP1 = fsin(du1dt, theta, phi, lMax)
    dv1dp = np.gradient(V1_tp, phi, axis=0)
    sin1 = fsin(dv1dp, theta, phi, lMax)
    divP2 = fsin(sin1, theta, phi, lMax)
    divP = divP1 + divP2 #\nabla^1 P
    divP_nm = projection(divP, theta, phi, lMax)
    p_nm = np.zeros((lMax, lMax), dtype=complex)
    for m in range (1, lMax):
        for n in range (m, lMax):
            p_nm[n,m] = (-1/(n*(n+1)))*divP_nm[n,m]
    return p_nm

In [9]:
def B_np(n,m):
    B_np = -n*(n-m+1)/(2*n+1)
    return B_np

In [10]:
def B_nm(n,m):
    B_nm = (n+1)*(n+m)/(2*n+1)
    return B_nm

In [11]:
def g(n,m):
    g_nm = ((2*n+1)*factorial(n+m)/factorial(n-m))**(1/2)
    return g_nm

In [12]:
#P_n^m, degree n, order m
#P_n^m(x), this is working and has been tested
def Pmn(m,n,x):
    #Pmn shape is the same as theta for x=cos(theta)
    leg = legendre(n)
    P_n = leg(x)  #P_n(x)
    deriv = P_n 
    d = 0
    for i in range(m):
        #d = np.gradient(deriv,x)
        d = np.gradient(deriv)
        deriv = d
    pcos = ((-1)**m)*((1-x**2)**(m/2))*deriv
    return pcos

In [13]:
#P_n(x)
def P_n(x, n):
    le = legendre(n)
    pn = le(x)
    return pn 

In [14]:
def associated_legendre(m, n, x):
    Pnm1 = P_n(x, 0)
    Pn = P_n(x, 1)
    for i in range(n-1):
        Pnp1 = (n-m+1)**(-1)*((2*n+1)*Pn-(n+m)*Pnm1)
        Pnm1 = Pn
        Pn = Pnp1
    return Pnp1